<a href="https://colab.research.google.com/github/nullvoid-j/Genre_Predection/blob/master/Data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#mounting google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
import re
from os import listdir
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import numpy as np
import pandas as pd
import collections
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#funtion to clean text
def clean_data(text):
    text=text.lower() 
    ## remove html tags ##
    text = re.sub(r"<.*?>", " ", text )
    #removing punctuation marks and short froms
    text = re.sub(r"\n", " ", text)   
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)   
    text = re.sub(r"\s\s+", " ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r"e - mail", "email", text)
    return text

In [6]:
files=listdir('/content/drive/My Drive/Movie Scripts data/')
len(files)

2837

In [0]:
# print([item for item, count in collections.Counter(a).items() if count>1])

[1, 2, 5]


In [13]:
list_of_docs=[]
stops=set(stopwords.words('english'))
for file in files:
  filtered_files=[]
  file_data=open('/content/drive/My Drive/Movie Scripts data/'+file, 'r')
  data=clean_data(file_data.read())
  for w in data.split():
    if w.isalpha() and w not in stops:
      filtered_files.append(w)
    else:
      continue
  list_of_docs.append(filtered_files)
  file_data.close()
  print(file+' has been processed')

file_2644.txt has been processed
file_2645.txt has been processed
file_2639.txt has been processed
file_2647.txt has been processed
file_2649.txt has been processed
file_2646.txt has been processed
file_2652.txt has been processed
file_2654.txt has been processed
file_2651.txt has been processed
file_2650.txt has been processed
file_2653.txt has been processed
file_2648.txt has been processed
file_265.txt has been processed
file_2656.txt has been processed
file_2660.txt has been processed
file_2657.txt has been processed
file_2659.txt has been processed
file_266.txt has been processed
file_2663.txt has been processed
file_2658.txt has been processed
file_2655.txt has been processed
file_2661.txt has been processed
file_2662.txt has been processed
file_2671.txt has been processed
file_267.txt has been processed
file_2669.txt has been processed
file_2665.txt has been processed
file_2668.txt has been processed
file_2670.txt has been processed
file_2666.txt has been processed
file_2664.txt

In [0]:
# #training own w2v model
# W2V_model=Word2Vec(list_of_docs, min_count=5, size=200, workers=4, seed=1)
# W2V_model.save('self_trained_w2v.model')

# words=list(W2V_model.wv.vocab)
# print(len(words))

In [16]:
#getting pre trained embedding file
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-04-06 11:36:29--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.79.46
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.79.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  86.6MB/s    in 20s     

2020-04-06 11:36:49 (78.9 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [17]:
#using google trained model
w2v_google_model=KeyedVectors.load_word2vec_format('/root/input/GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
#now computing average vector for each script
doc_vec_list=[]
file_count=0
for doc in list_of_docs:
  doc_vec=np.zeros(300)
  no_of_words=0
  for word in doc:
    try:
      word_vec=w2v_google_model.wv[word]
      doc_vec += word_vec
      no_of_words += 1
    except:
      # print("There was an exception. Passing...")
      pass
  doc_vec=doc_vec/no_of_words
  doc_vec_list.append(doc_vec)
  file_count +=1
  print(file_count)


In [19]:
len(doc_vec_list)

2837

In [0]:
vec_df=pd.DataFrame(data=doc_vec_list)
file_name_df=pd.DataFrame(data=files)
vec_df=vec_df.join(file_name_df, rsuffix='file')

In [21]:
vec_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,0file
0,0.025406,0.043207,0.022018,0.108354,-0.023790,-0.012236,0.004705,-0.082845,0.066575,0.071665,-0.029615,-0.116364,-0.059800,0.017148,-0.077333,0.078503,0.048371,0.114236,-0.001857,-0.077558,-0.011976,0.037242,0.095513,0.001120,-0.002607,-0.026158,-0.080751,0.083981,0.031020,-0.027709,-0.003886,0.038687,-0.029943,-0.020138,-0.053155,-0.028145,0.027445,0.054104,0.018986,0.062447,...,0.008764,-0.034703,-0.013909,0.020841,0.082626,-0.038338,-0.035463,-0.057970,-0.021609,-0.032171,0.069184,0.053198,0.027067,0.077093,-0.048055,-0.071444,-0.072176,-0.067836,0.057983,-0.000046,-0.004279,0.038215,0.059295,0.010059,-0.013160,-0.052407,0.009708,0.024161,0.026781,-0.006100,0.045394,-0.106182,0.023891,-0.053418,-0.048542,-0.005223,-0.065825,-0.013113,0.038021,file_2644.txt
1,0.063581,0.042804,0.020896,0.066932,-0.048847,-0.026213,0.049913,-0.083013,0.085321,0.107426,-0.018879,-0.110843,-0.025045,0.019020,-0.091494,0.067978,0.035585,0.112051,0.014313,-0.069600,0.009353,0.073111,0.033148,-0.010925,0.040131,-0.071616,-0.063656,0.071011,0.051274,-0.025532,-0.050882,0.011555,-0.049797,0.001481,-0.014147,-0.066423,0.051010,0.020931,0.001091,0.060354,...,-0.005770,-0.053268,0.008261,0.035186,0.112647,-0.012045,-0.010222,-0.072869,-0.016732,-0.004177,0.094225,0.064765,0.013005,0.084233,-0.050192,-0.076048,-0.072826,-0.040456,0.014235,0.010592,-0.030606,0.019631,0.059885,0.031903,-0.004989,-0.066708,0.025880,0.018516,0.054912,-0.067826,0.023178,-0.098482,0.003231,-0.023312,-0.010919,0.014480,-0.051430,0.015439,0.009260,file_2645.txt
2,0.042341,0.044336,0.000939,0.087049,-0.069071,-0.028438,0.022111,-0.091769,0.095171,0.059125,-0.041301,-0.136174,-0.051798,0.003669,-0.077972,0.091346,0.050019,0.115418,0.010548,-0.092528,-0.004042,0.054975,0.057986,-0.023882,0.004176,-0.028739,-0.098253,0.067246,0.024380,-0.005811,-0.004836,0.008737,-0.052358,-0.003053,-0.009105,-0.039208,0.029433,0.047354,0.029598,0.049998,...,0.006014,-0.023116,0.016251,0.022953,0.104282,-0.001550,-0.022698,-0.065400,-0.034864,0.013987,0.055868,0.062985,0.034907,0.048684,-0.068349,-0.070454,-0.070380,-0.062928,0.043608,-0.013677,0.002130,0.025312,0.068260,0.023993,0.008921,-0.045314,0.027690,0.039623,0.040034,-0.057516,0.037862,-0.088229,0.036587,-0.055329,-0.041143,0.001686,-0.078792,0.003566,0.018725,file_2639.txt
3,0.022705,0.043932,-0.002761,0.110469,-0.068331,-0.026937,0.007080,-0.106831,0.076370,0.076637,-0.048834,-0.150140,-0.040290,-0.000755,-0.088493,0.080214,0.042197,0.131836,0.039564,-0.071337,-0.011187,0.046188,0.102320,-0.024983,0.011686,-0.013479,-0.105408,0.094364,0.058737,-0.026486,-0.036944,0.022392,-0.033963,-0.043661,-0.043725,-0.048051,0.055627,0.064136,0.021751,0.051263,...,-0.008047,-0.031809,-0.007691,0.015665,0.082849,-0.026193,-0.019646,-0.024410,-0.042433,-0.022654,0.092350,0.083727,0.059194,0.070188,-0.054121,-0.058042,-0.100361,-0.062149,0.074285,0.009650,-0.001475,0.042421,0.078242,0.011153,-0.022373,-0.056441,-0.018566,0.035688,0.041579,-0.015980,0.058941,-0.060056,0.028741,-0.076760,-0.067000,0.022327,-0.067695,-0.016873,0.029241,file_2647.txt
4,0.023306,0.012919,-0.010126,0.079620,-0.025725,-0.034334,0.031463,-0.077264,0.064328,0.069393,-0.059040,-0.126407,-0.031060,-0.011760,-0.112819,0.090020,0.069495,0.136041,0.030191,-0.117453,0.011835,0.077633,0.124218,-0.024486,-0.022852,-0.050375,-0.084321,0.092194,0.050496,-0.020918,-0.039897,0.049601,-0.038506,-0.024149,-0.040985,-0.037750,0.051197,0.070326,0.021872,0.074097,...,0.002717,-0.023109,-0.018815,0.013405,0.104279,-0.043574,-0.017912,-0.078996,-0.045030,-0.057220,0.076755,0.041025,0.041362,0.060461,-0.044402,-0.047567,-0.081240,-0.085921,0.048454,0.034406,-0.019997,0.054153,0.061006,0.024679,-0.052012,-0.017333,0.006556,0.02

In [0]:
train_df=pd.read_csv('/content/drive/My Drive/Movie_test_train_data/Train.csv')
train_df=pd.merge(train_df, vec_df, how='inner', left_on='File_Name',right_on='0file')
train_df.drop(columns='0file', axis=1, inplace=True)

In [0]:
#saving the vectorized form in pickle file
with open('/content/drive/My Drive/W2V_data_wo_stops.pkl','wb') as f:
  pickle.dump(train_df, f)

In [0]:
###################################### Simple model ###################################### 

In [0]:
X=train_df.iloc[:, 2:].values
Y=train_df.iloc[:, 1].values

In [0]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [65]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
clf=MLPClassifier(random_state=0).fit(x_train, y_train)
pred=clf.predict(x_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score
print(accuracy_score(y_test, pred)*100)
# print(precision_score(y_test, log_pred, average='weighted'))
# print(recall_score(y_test, log_pred, average='weighted'))

22.474747474747474


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
